# Query 1: What is the average prize money won by players in a season?

In [111]:
import pandas as pd
MatchHomeTeamInfo = pd.read_csv(r"E:\Data Analysis Course\Data analyse\tennis_data_analysis\data\raw\MatchHomeTeamInfo.csv")
MatchAwayTeamInfo = pd.read_csv(r"E:\Data Analysis Course\Data analyse\tennis_data_analysis\data\raw\MatchAwayTeamInfo.csv")
MatchSeasonInfo = pd.read_csv(r"E:\Data Analysis Course\Data analyse\tennis_data_analysis\data\raw\MatchSeasonInfo.csv")

# Merge MatchHomeTeamInfo with MatchSeasonInfo on 'match_id'
home_season_merged = pd.merge(MatchHomeTeamInfo, MatchSeasonInfo, on='match_id')

# Merge MatchAwayTeamInfo with MatchSeasonInfo on 'match_id'
away_season_merged = pd.merge(MatchAwayTeamInfo, MatchSeasonInfo, on='match_id')

# Concatenate the merged DataFrames
combined_df = pd.concat([home_season_merged[['current_prize', 'season_id']],
                         away_season_merged[['current_prize', 'season_id']]], 
                        ignore_index=True).drop_duplicates(subset= "season_id")

# Group by 'season_id' and calculate the average prize money
average_prize_money = combined_df.groupby('season_id')['current_prize'].mean().reset_index()

average_prize_money.columns = ['Season', 'Average_Prize_Money']

print(average_prize_money)



     Season  Average_Prize_Money
0     52016             104859.0
1     52017             143350.0
2     55516             229486.0
3     55518             138859.0
4     55542             254456.0
..      ...                  ...
155   60791               1124.0
156   60793                  0.0
157   60938               1655.0
158   60939               1513.0
159   60941                618.0

[160 rows x 2 columns]


# 2.Which match had the highest number of aces by a single player?

In [25]:
PeriodInfo = pd.read_csv(r"E:\Data Analysis Course\Data analyse\tennis_data_analysis\data\raw\PeriodInfo.csv")
max_aces_match = PeriodInfo[PeriodInfo['statistic_name'] == 'aces'].set_index('match_id')[['home_stat', 'away_stat']].max(axis=1).idxmax()
print(f"Match with the highest number of aces by a single player: Match ID {max_aces_match}")

Match with the highest number of aces by a single player: Match ID 12276774


# 3. What is the win/loss ratio of players against opponents from the same country?

In [110]:
MatchEventInfo = pd.read_csv(r"E:\Data Analysis Course\Data analyse\tennis_data_analysis\data\raw\MatchEventInfo.csv")

# Merge MatchEventInfo with MatchHomeTeamInfo and MatchAwayTeamInfo to get player IDs
merged_df = MatchEventInfo[['match_id', 'winner_code']].merge(MatchHomeTeamInfo[['match_id', 'player_id' ,'country']], on='match_id', suffixes=('', '_home'))
merged_df = merged_df.merge(MatchAwayTeamInfo[['match_id', 'player_id', 'country']], on='match_id', suffixes=('', '_away')).drop_duplicates(subset="match_id")

merged_df.rename(columns={'player_id': 'player_id_home', 'country': 'country_home'}, inplace=True)

# Filter matches where players are from the same country
same_country_matches = merged_df[merged_df['country_home'] == merged_df['country_away']]

# Calculate win/loss ratio
win_loss_ratio = same_country_matches.groupby('country_home')['winner_code'].apply(lambda x: (x == 1).sum() / (x == 2).sum()).reset_index(name='win_loss_ratio')
max_win_loss_ratio = win_loss_ratio.max()
max_win_loss_ratio_country = max_win_loss_ratio.iloc()

print("Win/Loss ratio of players against opponents from the same country:")
print(win_loss_ratio)

Win/Loss ratio of players against opponents from the same country:
      country_home  win_loss_ratio
0        Argentina        0.933333
1        Australia        4.000000
2          Austria        1.000000
3          Belgium        0.000000
4           Brazil             inf
5           Canada             inf
6            Chile             inf
7            China        0.729730
8   Chinese Taipei        0.000000
9         Colombia        1.000000
10         Croatia        0.000000
11  Czech Republic        0.750000
12           Egypt             inf
13          France        0.741935
14         Germany        0.538462
15           India        0.111111
16          Israel             inf
17           Italy        0.863636
18           Japan        1.275000
19      Kazakhstan        0.000000
20          Mexico        1.000000
21     Netherlands             inf
22            Peru        0.000000
23          Poland        0.000000
24        Portugal        0.000000
25         Romania     

C:\Users\SilkRoadit.com\AppData\Local\Temp\ipykernel_25836\1600514998.py:13: RuntimeWarning: divide by zero encountered in scalar divide
  win_loss_ratio = same_country_matches.groupby('country_home')['winner_code'].apply(lambda x: (x == 1).sum() / (x == 2).sum()).reset_index(name='win_loss_ratio')


# 4. which country has participated in the ATP tournaments the most?

In [109]:
MatchTournamentInfo = pd.read_csv(r"E:\Data Analysis Course\Data analyse\tennis_data_analysis\data\raw\MatchTournamentInfo.csv")

all_players = pd.concat([MatchHomeTeamInfo[['match_id', 'country']], MatchAwayTeamInfo[['match_id', 'country']]])
all_players_tournaments = all_players.merge(MatchTournamentInfo[["match_id" , "tournament_category_name"]], on='match_id' )

participations_by_country_ATP = all_players_tournaments[all_players_tournaments["tournament_category_name"] == "ATP"]

participations_by_country = participations_by_country_ATP['country'].value_counts()

# Get the country with the most participations
most_participations_country = participations_by_country.idxmax()
most_participations_count = participations_by_country.max()

print(f"Country with the most ATP tournament participations: {most_participations_country} ({most_participations_count} participations)")

Country with the most ATP tournament participations: France (688 participations)
